## SRJ

## Adapt - GCIM

## First step - To create Generating functions

## Generating fns code ready. Only single 

In [1]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)  

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    excitationlist.extend(singles)
    excitationlist.extend(doubles)
    print('Excitations are', excitationlist)
    theta = np.zeros(len(singles)+ len(doubles))
    theta[:] = 45
    print('thetas are', theta)
    for excitation in excitationlist:
        @qml.qnode(dev)
        def superposition():
            qml.BasisState(hf_state, wires=range(qubits))
            if len(excitation) == 2:
                print('Single Exc going in', excitation)
                qml.SingleExcitation(np.pi/4, wires=excitation)
            else:
                print('Double Exc of doubles going in', excitation)
                qml.DoubleExcitation(np.pi/4, wires=excitation)
            return qml.state()
        super = superposition()
        generatingfns.append(super)
        print('Generating functions are', generatingfns)
    print('len of Generating functions is', len(generatingfns))

In [2]:
adaptvqe(adapt_it=4, e_th=1e-12)

Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668924
Excitations are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7], [0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
thetas are [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.
 45. 45. 45. 45. 45. 45. 45. 45.]
Single Exc going in [0, 4]
Generating functions are [array([ 0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.       

## Included 2 single excitations at time

In [7]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)  

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    excitationlist.extend(singles)
    print('Excitations are', excitationlist)




    for i, j in itertools.combinations(excitationlist, 2):
        if len(i) == 2 and len(j) == 2:  # Ensure both are singles
            print(f'Applying excitations: {i} and {j}')
            @qml.qnode(dev)
            def superposition():
                qml.BasisState(hf_state, wires=range(qubits))
                qml.SingleExcitation(np.pi/4, wires=i)
                qml.SingleExcitation(np.pi/4, wires=j)
                return qml.state()                
            super= superposition()
            generatingfns.append(super)
            print('Generating functions are', super)
    print('len of Generating functions is', len(generatingfns))
        
adaptvqe(adapt_it=4, e_th=1e-12)

Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668922
Excitations are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]
Applying excitations: [0, 4] and [0, 6]
Generating functions are [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.